In [5]:
import sys
import os
sys.path.append('..')

In [6]:
import pdf2image
from ipywidgets import Layout


from utils.dataloader import Dataloader
from utils.parser import process_image
from utils.cluster import cluster_text
from utils.extract import extract_fields
from utils.rule_gen import generate_query

from models.document import Document
from models.spatial_text import Page, Line

import hyperwidget


def convert_page_to_dict(page_obj):
    page = {}
    page['width'] = page_obj.width
    page['height'] = page_obj.height
    page['lines'] = []
    for line in page_obj.lines:
        page['lines'].append(
            {
                'height': line.height, 'width': line.width,
                'left': line.left, 'top': line.top, 'text': str(line)
            }
        )
    return page

def get_docs():
    w2_sample_dir = os.path.join('../data', 'sample', 'w2')
    data_dir = os.path.join(w2_sample_dir, 'single_clean')
    label_path = os.path.join(w2_sample_dir, 'single_label.csv')
    return Dataloader(data_dir, label_path)
dl = get_docs()

In [7]:
label_page = dl.get_document(0).pages[0]
hyperview = hyperwidget.HyperWidget(
    page=convert_page_to_dict(label_page),
    layout=Layout(overflow_x='auto')
)

In [8]:
hyperview

HyperWidget(layout=Layout(overflow_x='auto'), line_idxs=[0], page={'width': 1228, 'height': 1636, 'lines': [{'…

In [48]:
print("Chosen Lines: ", hyperview.lines)
query = generate_query(label_page.lines[hyperview.line_idxs[0]], label_page)
print("Generated Query: ", query)

Chosen Lines:  [{'height': 28, 'width': 138, 'left': 131, 'top': 1456, 'text': '282-08-755'}]
Generated Query:  {'name': 'Extracted Field', 'arguments': {'x-position': 0.16286644951140064, 'y-position': 0.8985330073349633, 'entity': 'CARDINAL', 'word-neighbors': ['IN', '604-70-391', '_[86611.24', 'f', "Employee's", 'address', 'and', 'ZIP', 'code', 'Employers', 'state', 'ID', 'number', 'TX'], 'word-neighbor-top-thres': 0.05, 'word-neighbor-left-thres': 0.1}, 'weights': {'x-position': 0.5, 'y-position': 0.2, 'entity': 0.5, 'word-neighbors': 0.2}}


In [49]:
extract_fields(dl.get_document(0), [query])

{'Extracted Field': [field(idx=156, line="282-08-755", score=1.30),
  field(idx=161, line="604-70-391 _[86611.24", score=1.25)]}